<a href="https://colab.research.google.com/github/gonzalovaldenebro/NaturalLanguageProcessing-Portfolio/blob/main/F6_1_NeuralLanguageModeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<div style="display: flex; align-items: flex-start;">
  <div>
      <h1>CS 195: Natural Language Processing</h1>
      <h2>Neural Language Modeling</h2>
      </br>
    <a href="https://colab.research.google.com/github/ericmanley/f23-CS195NLP/blob/main/F6_1_NeuralLanguageModeling.ipynb">
      <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab">
    </a>
  </div>
  <div style="margin-left: 20px;">
    <img src="https://github.com/ericmanley/f23-CS195NLP/blob/main/images/dalle_neural_net_viz.png?raw=1" width="500" style="display: block;">
  </div>
</div>


**Cover Illustration:** generated by Dall E using the ChatGPT 4 interface, prompted for a visualization of the network used in the code below. *That's not what I meant.*

## Announcement

AI - English Faculty Candidate: Gabriel Ford

Meeting with students: Thursday at 3:30pm in Howard 309

Scholarly Presentation: Friday at 9:00am in Howard ???

## Reference

SLP: Neural Networks and Neural Language Models, Chapter 7 of Speech and Language Processing by Daniel Jurafsky & James H. Martin https://web.stanford.edu/~jurafsky/slp3/7.pdf



In [ ]:
import sys
!{sys.executable} -m pip install datasets keras tensorflow

## Dataset for today

AG News dataset
* short news articles
* four classes: World, Sports, Business, Sci/Tech

https://huggingface.co/datasets/ag_news


In [ ]:
from datasets import load_dataset
data = load_dataset("ag_news")

print(data["train"]["text"][0])

# 0 is World
# 1 is Sports
# 2 is Business
# 3 is Sci/Tech
print(data["train"]["label"][0])

Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\band of ultra-cynics, are seeing green again.
2


## Review: Text Classification with Keras

Last time, we saw
* how to do text classification when there are more than 2 classes
    - one hot encoded output layer, one node per class, *softmax* output
    - categorical crossentropy loss
* embedding layer
    - pad sequences to all be same length
    - learn vector for each word representing word semantics
    
<div>
    <img src="https://github.com/ericmanley/f23-CS195NLP/blob/main/images/neural_text_classification.png?raw=1">
</div>

image source: SLP Fig. 7.11, https://web.stanford.edu/~jurafsky/slp3/7.pdf

*pooling* combines/aggregates all of the embeddings in some way

In [ ]:
from datasets import load_dataset
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Embedding, Flatten, GlobalMaxPooling1D, GlobalAveragePooling1D
from datasets import load_dataset
import numpy as np


data = load_dataset("ag_news")

# Prepare the tokenizer and fit on the training text
tokenizer = Tokenizer()
tokenizer.fit_on_texts(data["train"]["text"])
vocabulary_size = len(tokenizer.word_index) + 1

# Convert text to sequence of integers
train_sequences = tokenizer.texts_to_sequences(data["train"]["text"])
test_sequences = tokenizer.texts_to_sequences(data["test"]["text"])

# Pad sequences to ensure uniform length; you can decide the max length based on your dataset's characteristics
max_length = 100  # This should be adjusted based on the dataset
train_encoding_array = pad_sequences(train_sequences, maxlen=max_length, padding='post')
test_encoding_array = pad_sequences(test_sequences, maxlen=max_length, padding='post')

# Convert labels to one-hot vectors
train_labels = data["train"]["label"]
test_labels = data["test"]["label"]
train_labels_array = to_categorical(train_labels, num_classes=4)
test_labels_array = to_categorical(test_labels, num_classes=4)

#create a neural network architecture
model = Sequential()
model.add(Embedding(input_dim=vocabulary_size, output_dim=50, input_length=max_length))
model.add(Flatten())
#use one of these instead of Flatten() to try a pooling method
#model.add(GlobalMaxPooling1D())
#model.add(GlobalAveragePooling1D())
model.add(Dense(20, input_dim=max_length, activation='relu'))
model.add(Dense(4, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(train_encoding_array, train_labels_array, epochs=10, verbose=1)

loss, accuracy = model.evaluate(test_encoding_array, test_labels_array)
print(f"Test accuracy: {accuracy*100:.2f}%")


Epoch 1/10
3750/3750 [==============================] - 168s 45ms/step - loss: 0.3211 - accuracy: 0.8834
Epoch 2/10
3750/3750 [==============================] - 164s 44ms/step - loss: 0.1137 - accuracy: 0.9621
Epoch 3/10
3750/3750 [==============================] - 159s 42ms/step - loss: 0.0316 - accuracy: 0.9907
Epoch 4/10
3750/3750 [==============================] - 168s 45ms/step - loss: 0.0158 - accuracy: 0.9962
Epoch 5/10
1980/3750 [==============>...............] - ETA: 1:24 - loss: 0.0094 - accuracy: 0.9977

KeyboardInterrupt: ignored

## Neural Language Modeling

**Neural Language Modeling:** predict next word(s) from previous ones - like what we did with Markov models

Like classification, but output is softmax of every possible word in the vocabulary

Often a first step before extending the model to do summarization, translation, dialog, and other NLP tasks

<div>
    <img src="https://github.com/ericmanley/f23-CS195NLP/blob/main/images/neural_language_modeling.png?raw=1">
</div>

image source: SLP Fig. 7.13, https://web.stanford.edu/~jurafsky/slp3/7.pdf

## A Neural Language Model in Keras

Let's start by sampling some data from our news dataset

Then split into a training and testing set

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, Flatten
from keras.utils import to_categorical
from keras.utils import pad_sequences
from keras.preprocessing.text import Tokenizer
from datasets import load_dataset
from sklearn.model_selection import train_test_split
import numpy as np
import random

data = load_dataset("ag_news")

data_subset, _ = train_test_split(data["train"]["text"],train_size=5000)
train_data, test_data = train_test_split(data_subset,train_size=0.8)

# Prepare the tokenizer and fit on the training text
tokenizer = Tokenizer()
tokenizer.fit_on_texts(data_subset)
vocabulary_size = len(tokenizer.word_index) + 1
print("Vocabulary size:",vocabulary_size)

# Convert text to sequences of integers
train_texts = tokenizer.texts_to_sequences(train_data)

Vocabulary size: 18479


## Preparing training examples

We want to take the sequences like

In [ ]:
print( train_texts[0] )

[2968, 115, 8, 10628, 6, 77, 18323, 858, 3130, 10628, 107, 24, 4859, 3250, 84, 44, 355, 1, 18324, 200, 139, 99, 316, 44, 31, 7782, 48, 79]


and slide a window across to predict the next word

In [ ]:
print("Use",train_texts[0][0:5],"to predict",train_texts[0][5])
print("Use",train_texts[0][1:6],"to predict",train_texts[0][6])
print("Use",train_texts[0][2:7],"to predict",train_texts[0][7])
print("Use",train_texts[0][3:8],"to predict",train_texts[0][8])
print("etc.")

Use [2968, 115, 8, 10628, 6] to predict 77
Use [115, 8, 10628, 6, 77] to predict 18323
Use [8, 10628, 6, 77, 18323] to predict 858
Use [10628, 6, 77, 18323, 858] to predict 3130
etc.


## Group Discussion

What data structures (lists, matrixes, etc.) do we need to prepare to make this a classification problem?


I will need a vector that has got one-hot encoded values

## Preparing all of the examples

In [ ]:
# Decide the sequence length
sequence_length = 5  # Length of the input sequence before predicting the next word

# Create the sequences
predictor_sequences = []
targets = []
for text in train_texts:
    for i in range(sequence_length, len(text)):
        # Take the sequence of tokens as input and the next token as target
        curr_target = text[i]
        curr_predictor_sequence = text[i-sequence_length:i]
        predictor_sequences.append(curr_predictor_sequence)
        targets.append(curr_target)


print("Number of sequences:",len(predictor_sequences))


print("First train text:",train_texts[0])
print("Example sequence 0:",predictor_sequences[0]," target:",targets[0])
print("Example sequence 1:",predictor_sequences[1]," target:",targets[1])
print("Example sequence 2:",predictor_sequences[2]," target:",targets[2])
print("Example sequence 3:",predictor_sequences[3]," target:",targets[3])
print("Example sequence 4:",predictor_sequences[4]," target:",targets[4])
print("Example sequence 5:",predictor_sequences[5]," target:",targets[5])


Number of sequences: 137889
First train text: [2968, 115, 8, 10628, 6, 77, 18323, 858, 3130, 10628, 107, 24, 4859, 3250, 84, 44, 355, 1, 18324, 200, 139, 99, 316, 44, 31, 7782, 48, 79]
Example sequence 0: [2968, 115, 8, 10628, 6]  target: 77
Example sequence 1: [115, 8, 10628, 6, 77]  target: 18323
Example sequence 2: [8, 10628, 6, 77, 18323]  target: 858
Example sequence 3: [10628, 6, 77, 18323, 858]  target: 3130
Example sequence 4: [6, 77, 18323, 858, 3130]  target: 10628
Example sequence 5: [77, 18323, 858, 3130, 10628]  target: 107


## Padding

Some of the sequences might be really short - so we'll pad them just in case.

In [ ]:
# Pad sequences to ensure uniform length
predictor_sequences_padded = pad_sequences(predictor_sequences, maxlen=sequence_length, padding='pre')

## The target output

Since we're making this into a classification problem, the output layer needs to have one node for each word in the vocabulary.

Target values need to be transformed into a one-hot encoded vector

In [ ]:

# Convert output to one-hot encoding
target_word_one_hot = to_categorical(targets, num_classes=vocabulary_size)

print("Predictors words 0:", predictor_sequences_padded[0])
print("target word 0:", targets[0])
print("target word 0 one hot encoded:", target_word_one_hot[0])

Predictors words 0: [ 2968   115     8 10628     6]
target word 0: 77
target word 0 one hot encoded: [0. 0. 0. ... 0. 0. 0.]


## Preparing the test set

We have to do all of those same things for the test set.

**Group Exercise:** Turn this into a function so that you can use it to prepare both the training and test sets.

In [ ]:
test_texts = tokenizer.texts_to_sequences(test_data)

# Create the sequences
predictor_sequences_test = []
targets_test = []
for text in test_texts:
    for i in range(sequence_length, len(text)):
        # Take the sequence of tokens as input and the next token as target
        curr_target = text[i]
        curr_predictor_sequence = text[i-sequence_length:i]
        predictor_sequences_test.append(curr_predictor_sequence)
        targets_test.append(curr_target)

# Pad sequences to ensure uniform length
predictor_sequences_padded_test = pad_sequences(predictor_sequences_test, maxlen=sequence_length, padding='pre')

# Convert target to one-hot encoding
target_word_one_hot_test = to_categorical(targets_test, num_classes=vocabulary_size)

## Making the above code a function

In [ ]:
def process_data(data,sequence,tokenizer):

  texts = tokenizer.texts_to_sequences(test_data)
  vocabulary_size = len(tokenizer.word_index) + 1

  # Create the sequences
  predictor_sequences = []
  targets = []
  for text in texts:
      for i in range(sequence_length, len(text)):
          # Take the sequence of tokens as input and the next token as target
          curr_target = text[i]
          curr_predictor_sequence = text[i-sequence_length:i]
          predictor_sequences.append(curr_predictor_sequence)
          targets.append(curr_target)

  # Pad sequences to ensure uniform length
  predictor_sequences_padded = pad_sequences(predictor_sequences, maxlen=sequence_length, padding='pre')

  # Convert target to one-hot encoding
  target_word_one_hot = to_categorical(targets, num_classes=vocabulary_size)

  return predictor_sequences_padded, target_word_one_hot

predictor_sequences_padded, target_word_one_hot = process_data(train_data, 5, tokenizer)
predictor_sequences_padded_test, target_word_one_hot_test = process_data(test_data,  5, tokenizer)

## Designing the Neural Network

We'll start with a simple network like the one we used for text classification.

In [ ]:
# Define the model
model = Sequential()
model.add(Embedding(input_dim=vocabulary_size, output_dim=50, input_length=sequence_length))
model.add(Flatten())
model.add(Dense(100, activation="relu"))
model.add(Dense(vocabulary_size, activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Fit the model - you can also pass in the test set
model.fit(predictor_sequences_padded, target_word_one_hot, epochs=10, verbose=1, validation_data=(predictor_sequences_padded_test, target_word_one_hot_test))

# The model can now be used to predict the next word in a sequence

Epoch 1/10
1073/1073 [==============================] - 65s 60ms/step - loss: 7.9410 - accuracy: 0.0467 - val_loss: 7.1632 - val_accuracy: 0.0548
Epoch 2/10
1073/1073 [==============================] - 57s 54ms/step - loss: 7.0340 - accuracy: 0.0681 - val_loss: 6.6100 - val_accuracy: 0.0821
Epoch 3/10
1073/1073 [==============================] - 57s 53ms/step - loss: 6.5412 - accuracy: 0.0897 - val_loss: 6.1386 - val_accuracy: 0.1108
Epoch 4/10
1073/1073 [==============================] - 55s 51ms/step - loss: 6.0977 - accuracy: 0.1168 - val_loss: 5.6959 - val_accuracy: 0.1451
Epoch 5/10
1073/1073 [==============================] - 56s 52ms/step - loss: 5.6573 - accuracy: 0.1525 - val_loss: 5.2234 - val_accuracy: 0.1905
Epoch 6/10
1073/1073 [==============================] - 56s 52ms/step - loss: 5.1675 - accuracy: 0.1981 - val_loss: 4.6730 - val_accuracy: 0.2477
Epoch 7/10
1073/1073 [==============================] - 55s 51ms/step - loss: 4.6114 - accuracy: 0.2525 - val_loss: 4.0559 -

## Testing the final model on the test set

In [ ]:
loss, accuracy = model.evaluate(predictor_sequences_padded_test, target_word_one_hot_test)
print(f"Test accuracy: {accuracy*100:.2f}%")

NameError: ignored

## Text Generation

We can use this model to successively generate words based on previous ones - like our Markov sequence on steroids.

Let's see how this works

In [ ]:
starter_string = "the government said that it"
tokens_list = tokenizer.texts_to_sequences([starter_string])
print(tokens_list)

tokens_array = np.array(tokens_list)
print(tokens_array)

[[1, 84, 19, 11, 18]]
[[ 1 84 19 11 18]]


the model will predict probabilities for each possible word in the output

In [ ]:
predicted_probabilities = model.predict(tokens_array,verbose=0)
print(predicted_probabilities)
print("We get a probability for each of the",len(predicted_probabilities[0]),"words")

[[7.7898148e-12 6.4446380e-05 6.8999807e-06 ... 7.7608224e-12
  7.1798600e-12 7.7865622e-12]]
We get a probability for each of the 18682 words


then we could get the word associated with the highest probability

In [ ]:
predicted_index = np.argmax(predicted_probabilities)
print("word index:",predicted_index)
predicted_word = tokenizer.index_word[predicted_index]
print("word:",predicted_word)

word index: 99
word: would


or you could generate a random word according the these probabilities (like with did with Markov text generation)

### putting it all together

In [ ]:
starter_string = "the government said that it"
tokens_list = tokenizer.texts_to_sequences([starter_string])
tokens = tokens_list[0]

for i in range(50):
    curr_seq = tokens[-sequence_length:]
    curr_array = np.array([curr_seq])
    predicted_probabilities = model.predict(curr_array,verbose=0)
    predicted_index = np.argmax(predicted_probabilities)
    predicted_word = tokenizer.index_word[predicted_index]
    print(predicted_word+" ",end="")
    tokens.append(predicted_index)


ValueError: ignored

## Applied Exploration

Pick another dataset and get it working with this code
* you will likely need to prepare the text a little differently - do you need to first break it into sentences?
* describe your dataset and what you did to prepare it

Perform a neural language modeling experiment
* experiment with something to try to find a well-performing model
    * sliding window size
    * number of hidden nodes in the network
    * learning rate
* describe what you did and write up an interpretation of your results

In [ ]:
from keras import optimizers

#an example on changing the learning rate
optimizer = optimizers.Adam(learning_rate=0.01)
model.compile(loss='categorical_crossentropy',optimizer=optimizer, metrics=["accuracy"])